In [1]:
from dataloader import TrajectoryDataset
from torch_geometric.loader import DataLoader
import torch

dataset = TrajectoryDataset('./datasets/eth/train')

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

batch_size = 2

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [10]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch.nn as nn
import numpy as np

class GraphConvs(torch.nn.Module):
	def __init__(self, num_node_features):
		super().__init__()
		
		self.conv1 = GCNConv(num_node_features, 64, improved=True)
		self.conv2 = GCNConv(64, 32, improved=True)
		self.conv3 = GCNConv(32, 16, improved=True)
		self.linear_conv = nn.Linear(16, 16)

	def forward(self, graph):
		x, edge_index = graph.x, graph.edge_index
		x = self.conv1(x, edge_index)
		x = F.relu(x)
		x = F.dropout(x, training=self.training)
		x = self.conv2(x, edge_index)
		x = F.relu(x)
		x = F.dropout(x, training=self.training)
		x = self.conv3(x, edge_index)
		x = self.linear_conv(x)
		return x


class GCN(torch.nn.Module):
	def __init__(self, num_node_features, max_peds, obs_len, pred_len, batch_size):
		super().__init__()

		self.max_peds = max_peds
		self.pred_len = pred_len
		self.obs_len = obs_len

		self.graph_convs = nn.ModuleList([GraphConvs(num_node_features) for _ in range(obs_len)])

		# trajectory prediction
		self.linear1 = nn.Linear(16, 16)
		self.linear2 = nn.Linear(16, max_peds * 2 * pred_len)

	def forward(self, sequence):
		graphs_out = [self.graph_convs[i](sequence[i]) for i in range(self.obs_len)]
		
		x = torch.concatenate(graphs_out)		
		x = self.linear1(x)
		x = F.relu(x)
		# x = F.dropout(x, training=self.training)
		x = self.linear2(x)
		x = F.relu(x)
		# x = F.dropout(x, training=self.training)
		x = x.reshape((self.max_peds, 2, self.pred_len, -1))
		return x

model = GCN(2, max_peds=5, obs_len=8, pred_len=12, batch_size=batch_size)


In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

# Training
for epoch in range(epochs):
	model.train(True)
	running_loss = 0.0

	for i, data in enumerate(trainloader, 0):
		input, target = data
		target = target.to(device)
		target = target.squeeze(axis=0)

		for j in range(len(input)):
			input[j] = input[j].to(device)

		# zero parameters gradients
		optimizer.zero_grad()

		output = model(input)
		loss = criterion(output.double(), target.double())
		loss.backward()

		optimizer.step()

		running_loss += loss.item()
	print(f'epoch:{epoch}, running_loss:{running_loss/i}')

cuda
torch.Size([10, 16])
torch.Size([5, 2, 12, 80])


RuntimeError: The size of tensor a (80) must match the size of tensor b (12) at non-singleton dimension 3

In [ ]:
data, target = next(iter(dataset))
for i in range(len(data)):
	data[i] = data[i].to(device)
target = target.to(device)

pred = model.forward(data)

import matplotlib.pyplot as plt

target = target.to('cpu').squeeze()
pred = pred.detach().to('cpu')

plt.plot(target[0][0], target[0][1])
plt.plot(pred[0][0], pred[0][1])

In [ ]:
max_peds = 5
obs_len = 8
print('how:',max_peds * obs_len * 16)